# Argilla

>[Argilla](https://argilla.io/) 是一个用于 LLM 的开源数据策展平台。
> 使用 Argilla，每个人都可以通过结合人工和机器反馈，更快地进行数据策展，从而构建强大的语言模型。我们支持 MLOps 周期中的每一个步骤，
> 从数据标注到模型监控。

<a target="_blank" href="https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/integrations/callbacks/argilla.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

在本指南中，我们将演示如何使用 `ArgillaCallbackHandler` 来跟踪您的 LLM 的输入和响应，以便在 Argilla 中生成数据集。

跟踪 LLM 的输入和输出来为将来的微调生成数据集非常有用。当您使用 LLM 为特定任务（如问答、摘要或翻译）生成数据时，这一点尤其有用。

## 安装和设置

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai argilla

### 获取 API 凭证

要获取 Argilla API 凭证，请按以下步骤操作：

1. 转到您的 Argilla UI。
2. 点击您的个人资料图片并转到“我的设置” (My settings)。
3. 然后复制 API 密钥 (API Key)。

在 Argilla 中，API URL 将与您的 Argilla UI 的 URL 相同。

要获取 OpenAI API 凭证，请访问 https://platform.openai.com/account/api-keys

In [11]:
import os

os.environ["ARGILLA_API_URL"] = "..."
os.environ["ARGILLA_API_KEY"] = "..."

os.environ["OPENAI_API_KEY"] = "..."

### 设置 Argilla

要使用 `ArgillaCallbackHandler`，我们需要在 Argilla 中创建一个新的 `FeedbackDataset` 来跟踪您的 LLM 实验。为此，请使用以下代码：

In [3]:
import argilla as rg

In [4]:
from packaging.version import parse as parse_version

if parse_version(rg.__version__) < parse_version("1.8.0"):
    raise RuntimeError(
        "`FeedbackDataset` is only available in Argilla v1.8.0 or higher, please "
        "upgrade `argilla` as `pip install argilla --upgrade`."
    )

In [ ]:
dataset = rg.FeedbackDataset(
    fields=[
        rg.TextField(name="prompt"),
        rg.TextField(name="response"),
    ],
    questions=[
        rg.RatingQuestion(
            name="response-rating",
            description="How would you rate the quality of the response?",
            values=[1, 2, 3, 4, 5],
            required=True,
        ),
        rg.TextQuestion(
            name="response-feedback",
            description="What feedback do you have for the response?",
            required=False,
        ),
    ],
    guidelines="You're asked to rate the quality of the response and provide feedback.",
)

rg.init(
    api_url=os.environ["ARGILLA_API_URL"],
    api_key=os.environ["ARGILLA_API_KEY"],
)

dataset.push_to_argilla("langchain-dataset")

> 📌 注意：目前，只有 prompt-response 配对被支持为 `FeedbackDataset.fields`，所以 `ArgillaCallbackHandler` 只会跟踪 prompt（即 LLM 输入）和 response（即 LLM 输出）。

## 追踪

您可以使用以下代码来使用 `ArgillaCallbackHandler`，或者直接使用后面章节中的任一示例。

In [ ]:
from langchain_community.callbacks.argilla_callback import ArgillaCallbackHandler

argilla_callback = ArgillaCallbackHandler(
    dataset_name="langchain-dataset",
    api_url=os.environ["ARGILLA_API_URL"],
    api_key=os.environ["ARGILLA_API_KEY"],
)

### 场景 1：跟踪一个 LLM

首先，我们只运行一个 LLM 几次，并将生成的提示-响应对捕获到 Argilla 中。

In [7]:
from langchain_core.callbacks.stdout import StdOutCallbackHandler
from langchain_openai import OpenAI

argilla_callback = ArgillaCallbackHandler(
    dataset_name="langchain-dataset",
    api_url=os.environ["ARGILLA_API_URL"],
    api_key=os.environ["ARGILLA_API_KEY"],
)
callbacks = [StdOutCallbackHandler(), argilla_callback]

llm = OpenAI(temperature=0.9, callbacks=callbacks)
llm.generate(["Tell me a joke", "Tell me a poem"] * 3)

LLMResult(generations=[[Generation(text='\n\nQ: What did the fish say when he hit the wall? \nA: Dam.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe Moon \n\nThe moon is high in the midnight sky,\nSparkling like a star above.\nThe night so peaceful, so serene,\nFilling up the air with love.\n\nEver changing and renewing,\nA never-ending light of grace.\nThe moon remains a constant view,\nA reminder of life’s gentle pace.\n\nThrough time and space it guides us on,\nA never-fading beacon of hope.\nThe moon shines down on us all,\nAs it continues to rise and elope.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nQ. What did one magnet say to the other magnet?\nA. "I find you very attractive!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text="\n\nThe world is charged with the grandeur of God.\nIt will flame out, like shining from shook foil;\nIt gathers to a greatness, like t

![Argilla UI with LangChain LLM input-response](https://docs.argilla.io/en/latest/_images/llm.png)

### 场景 2：跟踪链式 LLM

然后，我们可以使用提示模板创建一个链，并在 Argilla 中跟踪初始提示和最终响应。

In [8]:
from langchain.chains import LLMChain
from langchain_core.callbacks.stdout import StdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

argilla_callback = ArgillaCallbackHandler(
    dataset_name="langchain-dataset",
    api_url=os.environ["ARGILLA_API_URL"],
    api_key=os.environ["ARGILLA_API_KEY"],
)
callbacks = [StdOutCallbackHandler(), argilla_callback]
llm = OpenAI(temperature=0.9, callbacks=callbacks)

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callbacks=callbacks)

test_prompts = [{"title": "Documentary about Bigfoot in Paris"}]
synopsis_chain.apply(test_prompts)



> Entering new LLMChain chain...
Prompt after formatting:
You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: Documentary about Bigfoot in Paris
Playwright: This is a synopsis for the above play:

> Finished chain.


[{'text': "\n\nDocumentary about Bigfoot in Paris focuses on the story of a documentary filmmaker and their search for evidence of the legendary Bigfoot creature in the city of Paris. The play follows the filmmaker as they explore the city, meeting people from all walks of life who have had encounters with the mysterious creature. Through their conversations, the filmmaker unravels the story of Bigfoot and finds out the truth about the creature's presence in Paris. As the story progresses, the filmmaker learns more and more about the mysterious creature, as well as the different perspectives of the people living in the city, and what they think of the creature. In the end, the filmmaker's findings lead them to some surprising and heartwarming conclusions about the creature's existence and the importance it holds in the lives of the people in Paris."}]

![Argilla UI with LangChain Chain input-response](https://docs.argilla.io/en/latest/_images/chain.png)

### 场景 3：使用带有工具的 Agent

最后，作为一种更高级的工作流，您可以创建一个使用一些工具的 Agent。这样 `ArgillaCallbackHandler` 将跟踪输入和输出，但不会跟踪中间步骤/思考过程，因此对于给定的提示，我们记录原始提示和对该给定提示的最终响应。

> 请注意，在此场景中我们将使用 Google Search API (Serp API)，因此您需要执行以下操作：
>
> 1.  使用 `pip install google-search-results` 安装 `google-search-results`。
> 2.  将 Serp API 密钥设置为 `os.environ["SERPAPI_API_KEY"] = "..."`（您可以在 https://serpapi.com/dashboard 找到它）。
>
> 否则，以下示例将无法工作。

In [10]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_core.callbacks.stdout import StdOutCallbackHandler
from langchain_openai import OpenAI

argilla_callback = ArgillaCallbackHandler(
    dataset_name="langchain-dataset",
    api_url=os.environ["ARGILLA_API_URL"],
    api_key=os.environ["ARGILLA_API_KEY"],
)
callbacks = [StdOutCallbackHandler(), argilla_callback]
llm = OpenAI(temperature=0.9, callbacks=callbacks)

tools = load_tools(["serpapi"], llm=llm, callbacks=callbacks)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callbacks=callbacks,
)
agent.run("Who was the first president of the United States of America?")



> Entering new AgentExecutor chain...
 I need to answer a historical question
Action: Search
Action Input: "who was the first president of the United States of America" 
Observation: George Washington
Thought: George Washington was the first president
Final Answer: George Washington was the first president of the United States of America.

> Finished chain.


'George Washington was the first president of the United States of America.'

![Argilla UI with LangChain Agent input-response](https://docs.argilla.io/en/latest/_images/agent.png)